In [40]:
import DataProvider
prov = DataProvider.DataProvider()

[new value of  12
---new value of  13
-new value of  14
-new value of  29
---new value of  32
---new value of  38
---------------new value of  42
----------------------------------------------new value of  43
------------------------new value of  70
-----

/Users/shota/ML_Final_project/DataProvider.py:135: RuntimeWarning: invalid value encountered in true_divide
  spectrogram = (spectrogram - mean) / std


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------new value of  72
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------]
[#########################################################################################################################################################################################################################################################################################################################################################################################################

In [41]:
import numpy as np
X = list()
y = list()
trainVal = list()

In [42]:
while prov.hasNext(True):
    Xt, yt = prov.next(True)
    for i in range(len(yt)):
        if yt[i] == 1:
            trainVal.append(i+1)
            
    X.append(Xt)
    y.append(yt)

In [43]:
X = np.nan_to_num(np.matrix(X))
y = np.matrix(y)

X.shape, y.shape

((688, 1440), (688, 5))

In [44]:
X2 = list()
y2 = list()
actualVal = list()

In [45]:
while prov.hasNext(False):
    X2t, y2t = prov.next(False)
    actualVal.append(y2t)
    
    k = np.zeros(5)
    k[y2t - 1] = 1
    
    X2.append(X2t)
    y2.append(k)

In [46]:
X2 = np.nan_to_num(np.matrix(X2))
y2 = np.matrix(y2)

X2.shape, y2.shape

((5, 1440), (5, 5))

In [47]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [48]:
model = Sequential()

In [49]:
model.add(Dense(units=64, activation='relu', input_dim = prov.returnSizeOfEverySpectogram()))
model.add(Dense(128))
model.add(Dense(32))
model.add(Dense(units=5, activation='softmax'))

In [50]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=200, batch_size=128)
#model.train_on_batch(X, y)

Epoch 1/200
688/688 [==============================] - 2s 2ms/step - loss: 1.7292 - accuracy: 0.1904
Epoch 2/200
688/688 [==============================] - 0s 223us/step - loss: 1.6209 - accuracy: 0.2645
Epoch 3/200
688/688 [==============================] - 0s 160us/step - loss: 1.5315 - accuracy: 0.3503
Epoch 4/200
688/688 [==============================] - 0s 426us/step - loss: 1.4555 - accuracy: 0.4244
Epoch 5/200
688/688 [==============================] - 0s 371us/step - loss: 1.3886 - accuracy: 0.4782
Epoch 6/200
688/688 [==============================] - 0s 381us/step - loss: 1.3253 - accuracy: 0.5189
Epoch 7/200
688/688 [==============================] - 0s 340us/step - loss: 1.2668 - accuracy: 0.5654
Epoch 8/200
688/688 [==============================] - 0s 210us/step - loss: 1.2122 - accuracy: 0.6061
Epoch 9/200
688/688 [==============================] - 0s 459us/step - loss: 1.1609 - accuracy: 0.6265
Epoch 10/200
688/688 [==============================] - 0s 207us/step - los

In [ ]:
loss_and_metrics = model.evaluate(X2, y2, batch_size=128)

In [ ]:
pred = model.predict(X2, batch_size=128)
pred

In [ ]:
pr = np.argmax(pred, axis=1)+1
pr

In [ ]:
correct = [1 if a == b else 0 for (a, b) in zip(pr, actualVal)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ("Accuracy: " + str(accuracy * 100) + "%")

In [ ]:
for i in pred:
    count = sum(i)
    for j in range(len(i)):
        i[j] = i[j] / count

In [ ]:
import copy
countGrade = copy.deepcopy(actualVal)
countGrade

In [ ]:
for i in range(len(countGrade)):
    ans = actualVal[i]
    countGrade[i] = pred[i][ans-1]

In [ ]:
result = sum(countGrade) / len(countGrade)
result

In [ ]:
model.summary()